<a href="https://colab.research.google.com/github/jmoro0408/sloped_pipe_volume/blob/main/working_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

* tidy code

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import HBox, VBox, widgets, Label, Layout
from IPython.display import display, Math
import warnings
pd.options.mode.chained_assignment = None  # default='warn' #disable pandas companing about creating a copy of a df
#warnings.filterwarnings('ignore')

In [2]:
#@title Calculation Inputs
box_text_style = {'description_width': 'initial'}

units = widgets.RadioButtons(
 options=["Metric", "Imperial"],
 value = "Metric",
 description="Units", 
 layout = Layout(width = "15%", height = "60px"),
 )

inital_grade_percent= widgets.BoundedFloatText(
    value=1,
    min=0,
    max=25,
    step=0.5,
    description = "Grade Percent ",
    style = box_text_style,
    layout = Layout(width = "25%", height = "30px"),
    disabled=False, 
)

max_grade_percent= widgets.BoundedFloatText(
    value=5,
    min=0,
    max=90,
    step=0.5,
    description = "Max Grade Percent (to plot) ",
    style = box_text_style,
    layout = Layout(width = "25%", height = "30px"),
    disabled=False, 
)

pipe_invert_elev=widgets.BoundedFloatText(
    value=-5,
    min=-1e6,
    max=1e6,
    step=0.1,
    description = "Pipe Invert Elevation (Geodetic)",
    style = box_text_style,
    layout = Layout(width = "15%", height = "30px"),
    disabled=False, 
)
pipe_diameter=widgets.BoundedFloatText(
    value=2,
    min=0,
    max=1e6,
    step=0.1,
    description = "Pipe Diameter",
    style = box_text_style,
    layout = Layout(width = "15%", height = "30px"),
    disabled=False, 
)
pipe_length= widgets.BoundedFloatText(
    value=200,
    min=0,
    max=1e6,
    step=0.1,
    description = "Pipe Length",
    style = box_text_style,
    layout = Layout(width = "15%", height = "30px"),
    disabled=False,
)
lower_liquid_level= widgets.BoundedFloatText(
    value=-3.75,
    min=-1e6,
    max=1e6,
    step=0.1,
    description = " Lower Liquid Elevation (Geodetic)",
    style = box_text_style,
    layout = Layout(width = "15%", height = "30px"),
    disabled=False, 
)

# box0 = HBox([units, inital_grade_percent])
# box1 = HBox([pipe_invert_elev, pipe_diameter ])
# box2 = HBox([pipe_length, lower_liquid_level])
VBox([units,inital_grade_percent,max_grade_percent, pipe_invert_elev, pipe_diameter,pipe_length, lower_liquid_level  ])


In [3]:
# initial values
pipe_invert_start = pipe_invert_elev.value #pipe invert at pipe end
pipe_diameter = pipe_diameter.value
initial_grade_percent = inital_grade_percent.value #inital grade value
pipe_length = pipe_length.value
lower_liquid_level = lower_liquid_level.value #liquid level elevation at pipe end. this is NOT the relative height of water above the pipe invert
maximum_grade_value = 5 #maximum grade (%) to plot 

In [4]:
units_dict = {
    "Metric":["m", "m3"],
    "Imperial":["ft", "ft3"]
}

units_length = units_dict[units.value][0]
units_volume = units_dict[units.value][1]

# Calculation


In [5]:

no_points_to_plot = 100
no_grades = np.arange(0,maximum_grade_value, 20 )
pipe_chainage = np.linspace(0, pipe_length, no_points_to_plot).tolist()
pipe_chainage = [round(elem) for elem in pipe_chainage]
relative_lower_liquid_level = np.abs(pipe_invert_start - lower_liquid_level)
if relative_lower_liquid_level > pipe_diameter:
  raise ValueError("Liquid level must be equal to or less than pipe diameter") 

In [6]:

def calculate_tilt_angle(initial_grade_percent):
  tilt_angle = np.arctan(initial_grade_percent / 100)
  tilt_angle = np.degrees(tilt_angle)
  
  return tilt_angle

def calculate_pipe_upper_invert(pipe_invert_elev,tilt_angle,pipe_length):
  pipe_upper_invert = np.tan(np.radians(tilt_angle)) * pipe_length + pipe_invert_elev
  return pipe_upper_invert

def calculate_upper_liquid_level(lower_liquid_level, tilt_angle, pipe_length):
  upper_liquid_level = lower_liquid_level
  return upper_liquid_level

def calculate_total_pipe_volume(pipe_diameter, pipe_length):
  pipe_cross_sectional_area = np.pi / 4 * (pipe_diameter**2)
  total_volume = pipe_cross_sectional_area * pipe_length
  total_volume = round(total_volume, 2)
  return total_volume

def calculate_sloped_pipe_volume(
    pipe_diameter, tilt_angle, pipe_length, relative_lower_liquid_level
):
    if tilt_angle == 0:
        tilt_angle = 0.001  # angle of zero breaks the np.tan function, so making a close approximation
    pipe_radius = pipe_diameter / 2
    tilt_angle_radians = np.radians(tilt_angle)
    _h0 = relative_lower_liquid_level - (pipe_length * np.tan(tilt_angle_radians))

    # "normal" case, where both ends have some liquid
    if _h0 >= 0:
        _lf = pipe_length

    # upper end has no liquid - liquid does not reach end of pipe
    elif _h0 < 0:
        # lower_liquid_level = 0
        _lf = pipe_length + _h0 / np.tan(tilt_angle_radians)
        _h0 = 0
        # assert _lf < pipe_length, "Liquid fill length cannot be greater than pipe length"

    _h1 = _h0 + _lf * np.tan(tilt_angle_radians)
    full_volume = None
    # Case where pipe is (at least) partially completely full.
    if _h1 > pipe_diameter:
        _lt = (_h1 - pipe_diameter) / np.tan(tilt_angle_radians)
        full_volume = np.pi * pipe_radius ** 2 * _lt
    else:
        _lt = 0

    _pipe_length = _lf - _lt

    _k = 1 - (_h0 / pipe_radius)
    _c = _k - ((_pipe_length * np.tan(tilt_angle_radians)) / pipe_radius)

    assert -1 <= _k <= 1, "k out of range"
    assert -1 <= _c <= 1, "c out of range"

    integral0 = pipe_radius ** 3 / np.tan(tilt_angle_radians)
    integral1 = _k * np.arccos(_k)
    integral2 = (1 / 3) * (np.sqrt(1 - (_k ** 2))) * (_k ** 2 + 2)
    integral3 = _c * np.arccos(_c)
    integral4 = (1 / 3) * (np.sqrt(1 - (_c ** 2))) * (_c ** 2 + 2)
    volume = integral0 * (integral1 - integral2 - integral3 + integral4)

    if (pipe_radius == relative_lower_liquid_level) and (
        tilt_angle == 0
    ):  # Pipe is exactly half full and no slope
        volume = (pipe_length * np.pi * pipe_radius ** 2) / 2
    if full_volume is not None:
        volume = volume + full_volume

    return volume

In [7]:
tilt_angle = calculate_tilt_angle(initial_grade_percent)
upper_pipe_invert = calculate_pipe_upper_invert(pipe_invert_start,tilt_angle,pipe_length)
upper_liquid_level = calculate_upper_liquid_level(lower_liquid_level, tilt_angle, pipe_length)

In [8]:
#create x points along the chainage
pipe_invert_points = np.linspace(pipe_invert_start, upper_pipe_invert, no_points_to_plot)

In [9]:
#initial_grade_percent_list = np.linspace(0, maximum_grade_value,no_points_to_plot ) #this will be the slider eventually
grade_step = maximum_grade_value**2/100
initial_grade_percent_list = np.arange(0, maximum_grade_value+grade_step, step = grade_step )
initial_grade_percent_list = np.round(initial_grade_percent_list, 2)

In [10]:
# Make sure chosen grade is actually included in list
if initial_grade_percent not in initial_grade_percent_list:
  initial_grade_percent_list = np.append(initial_grade_percent_list, initial_grade_percent)
  initial_grade_percent_list.sort()

In [11]:
#Building a dictionary that holdes the grade and every respective pipe invert y coordinate 
pipe_invert_dict = {}
for grade in initial_grade_percent_list:
  _tilt_angle = calculate_tilt_angle(grade)
  _upper_pipe_invert = calculate_pipe_upper_invert(pipe_invert_start, _tilt_angle,pipe_length)
  _pipe_invert_points = np.linspace(pipe_invert_start, _upper_pipe_invert, no_points_to_plot)
  _pipe_invert_points = np.round(_pipe_invert_points, 2)
  _pipe_invert_points = _pipe_invert_points.tolist()
  pipe_invert_dict.update({grade:_pipe_invert_points})


In [12]:
#Building a dictionary that holdes the grade and every respective pipe crown y coordinate 
# The pipe crown is just the invert + the pipe diameter, so this is easy to build from the invert dict
pipe_crown_dict = {}
for key, values in pipe_invert_dict.items():
  _crown_value = [value + pipe_diameter for value in values]
  _crown_value = [round(value, 2) for value in _crown_value]
  pipe_crown_dict.update({key:_crown_value})


In [13]:
# Building a dict to hold the respective upper liquid levels at every grade. This is just equal to the lower level as the water level does not move
liquid_level_dict = {}
for grade in initial_grade_percent_list:
  liquid_level_dict.update({grade: [lower_liquid_level] * no_points_to_plot})

# for grade in initial_grade_percent_list:
#   _tilt_angle = calculate_tilt_angle(grade)
#   _upper_liquid_level = calculate_upper_liquid_level(lower_liquid_level, _tilt_angle, pipe_length)
#   _upper_liquid_level_points = np.linspace(lower_liquid_level,_upper_liquid_level,  no_points_to_plot)
#   _upper_liquid_level_points = _upper_liquid_level_points.tolist()
#   liquid_level_dict.update({grade:_upper_liquid_level_points})



In [14]:
"""this one is tricky. Trying to make sure any point where the liquid level is below the pipe invert is not plotted. 

I think there is a neater way to do this but I tried going directly to the go.Figure() object, and to the individual dicts but
got lost navigating the nested stucture. This way works for now. 
"""

pipe_invert_df = pd.DataFrame.from_dict(pipe_invert_dict) #create dataframes from both dicts
liquid_level_df = pd.DataFrame.from_dict(liquid_level_dict)

for column in pipe_invert_df:
  for index in pipe_invert_df.index:
    if liquid_level_df[column].iloc[index] < pipe_invert_df[column].iloc[index]: # if liquid level < pipe invert, replace with NaN
      liquid_level_df[column].iloc[index] = np.nan

liquid_level_dict = liquid_level_df.to_dict(orient = "list")

del pipe_invert_df, liquid_level_df #dont need these dataframes anymore

In [15]:
#Building a doct that holds the pipe volume for each grade %
liquid_volume_grade_dict = {}
for key, value in liquid_level_dict.items():
  _pipe_radius = pipe_diameter/2
  _tilt_angle = calculate_tilt_angle(key)
  _pipe_volume = calculate_sloped_pipe_volume(pipe_diameter, _tilt_angle, pipe_length, relative_lower_liquid_level)
  liquid_volume_grade_dict.update({key:_pipe_volume})


liquid_volume_grade_dict = {key: round(liquid_volume_grade_dict[key], 2) for key in liquid_volume_grade_dict} # rouding grades for plotting

In [16]:
pipe_total_volume = calculate_total_pipe_volume(pipe_diameter, pipe_length)

In [17]:
air_volume_dict = {} # Volume of air is just toal volume - volume of liquid
for key, value in liquid_volume_grade_dict.items():
  air_volume_dict.update({key: pipe_total_volume-value})
  air_volume_dict = {key: round(air_volume_dict[key], 2) for key in air_volume_dict}

In [18]:
air_liquid_percent_dict = {}
for key, value in liquid_volume_grade_dict.items():
  _liquid_percent = (liquid_volume_grade_dict[key] / pipe_total_volume) *100
  _liquid_percent = round(_liquid_percent,2)
  _air_percent = 100 - _liquid_percent
  _air_percent = round(_air_percent, 2)
  air_liquid_percent_dict.update({key: [_liquid_percent, _air_percent]})


In [19]:
# two lists of air percents and liquid percents makes plotting a whole easier, so just use list comprehension to explode the dict
air_percents_list = [list(air_liquid_percent_dict.values())[i][1] for i in range(len(air_liquid_percent_dict.values()))]
liquid_percents_list = [list(air_liquid_percent_dict.values())[i][0] for i in range(len(air_liquid_percent_dict.values()))]


# Single value output

In [20]:
display(Math(r'\Large Liquid \: volume: {} \: {}'.format(liquid_volume_grade_dict[initial_grade_percent], units_volume)))

<IPython.core.display.Math object>

The following cells show how the grade % affects the liquid volume. 

# Plotting

In [21]:
fig1 = go.Figure()
trace_list1 = []
trace_list2 = []
trace_list3 = []
for grade in pipe_invert_dict.keys():
  _tilt_angle = round(calculate_tilt_angle(grade),2)
  trace_list1.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=pipe_invert_dict[grade], 
                                name = "Pipe Invert",
                                showlegend=False, 
                                line=dict(color='gray', width=3)))
  
  trace_list2.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=pipe_crown_dict[grade], 
                                name = "Pipe Crown",
                                showlegend=False, 
                                line=dict(color='gray', width=3)))
  
  trace_list3.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=liquid_level_dict[grade], 
                                name = "Liquid Level", 
                                line=dict(color='royalblue', width=2)))

In [27]:
{
    "tags": [
        "hide-input",
    ]
}
#Building the slider
# in order to capture both the crown and invert moving together with a singler slider step we need to append both traces to a single list

num_steps = len(initial_grade_percent_list) #each step = a different grade
slider_start_value = list(np.where(initial_grade_percent_list ==initial_grade_percent))[0][0]


#have to be really careful here. You want the slider to be aware of the ordering of traces, see https://community.plotly.com/t/multiple-traces-with-a-single-slider-in-plotly/16356

fig1 = go.Figure(data = trace_list1 + trace_list2+trace_list3)

fig1.data[0].visible = True #this is the pipe invert data
fig1.data[num_steps].visible = True #pipe crown data
fig1.data[num_steps*2].visible = True #liquid level data


# Set initial slider/title index
start_index = 0

steps = []
for i in range(num_steps):
  current_grade_value = round(initial_grade_percent_list[i], 2)
  step = dict(
      method="update", 
      args = [
              {"visible":[False] * len(fig1.data)}, 
              {"title":f"Liquid Level Along Pipe at {current_grade_value} % Grade, Liquid Volume: {liquid_volume_grade_dict[current_grade_value]} {units_volume}"}, 
              ], 
              label=str(round(list(pipe_invert_dict.keys())[i],2)))
  
  step["args"][start_index]["visible"][i] = True  # Toggle i'th trace to "visible" - pipe invert
  step["args"][start_index]["visible"][i+num_steps] = True #pipe crown
  step["args"][start_index]["visible"][i+num_steps*2] = True #liquid level
  steps.append(step)

sliders = [dict( 
    active=slider_start_value,
    currentvalue={"prefix": "Grade: ", "suffix": " %"},
    pad={"t": 50},
    steps=steps
)]



fig1.update_layout(
    template = "simple_white", 
    title=f"Liquid Level Along Pipe at {start_index} % Grade, Liquid Volume: {liquid_volume_grade_dict[start_index]}",
    xaxis_title=f"Pipe Chainage ({units_length})",
    yaxis_title="Elevation (mGeodetic)",
    sliders=sliders,
  #Legend info 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
)

fig1.update_yaxes(range=[min(min(pipe_invert_dict.values()))-1,max(max(pipe_crown_dict.values()))]) #have to lock the y axis, otherwise the axis just updates and the line doesnt "move"
fig1.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across")
fig1.update_yaxes(showspikes=True,spikecolor="green")

fig1.show()



In [23]:
# Plotting 
fig2_height =500
fig2_graph_width = 0.65


fig2 = make_subplots(rows = 1, cols = 2, 
                     column_widths = [fig2_graph_width, 1-fig2_graph_width],
                     horizontal_spacing = 0.1,
                     specs=[[{"secondary_y":True,"type":"xy"}, {"secondary_y":False, "type":"table"}]]) #"specs" arg is tricky. in future go straight to help(make_subplots)


# Air/Volume Graph Fig
fig2.add_trace(go.Scatter(x=list(air_liquid_percent_dict.keys()), 
                          y= air_percents_list, 
                          mode='lines', 
                          name = "%  Air", 
                          showlegend=True, 
                          line=dict(color='orange', width=2)), 
               row=1, col=1)

fig2.add_trace(go.Scatter(x=list(air_liquid_percent_dict.keys()), 
                          y= liquid_percents_list,
                          mode='lines', 
                          name = "% Liquid", 
                          showlegend=True, 
                          line=dict(color='royalblue', width=2)), 
               row=1, col=1)

fig2.add_trace(go.Scatter(x=list(air_volume_dict.keys()), 
                          y= list(air_volume_dict.values()),
                          mode='lines', 
                          name = "Volume air",
                          showlegend=True, 
                          line=dict(color='orange', width=2, dash = "dot")), 
               row=1, col=1,
               secondary_y = True)

fig2.add_trace(go.Scatter(x=list(liquid_volume_grade_dict.keys()), 
                          y= list(liquid_volume_grade_dict.values()),
                          mode='lines', 
                          name = "Volume liquid",
                          showlegend=True, 
                          line=dict(color='royalblue', width=2, dash = "dot")), 
               row=1, col=1,
               secondary_y = True)

#Adding a line at user chosen grade
vline = [ (initial_grade_percent, initial_grade_percent), 
        (0, max(liquid_volume_grade_dict.values()))]

fig2.add_trace(go.Scatter(x=vline[0], 
                          y= vline[1],
                          mode='lines', 
                          name = "Chosen Grade",
                          opacity=0.5,
                          showlegend=False, 
                          line=dict(color='green', width=2, dash = "dot")), 
               row=1, col=1,
               secondary_y = False)

#Table Fig
header_color = 'lightslategray'
row_even_color = 'lightgrey'
row_odd_color = 'darkgray'
linecolor = "white"

header_list = ["Grade (%)", f"Liquid ({units_volume})", f"Air ({units_volume})", "Liquid (%)", "Air (%)"]

#highlighting the initally selected row. Note the double list for rows. Single list seems to sort by columns. 

fill_color = [["lightgrey" if grade != initial_grade_percent else "dodgerblue" for grade in liquid_volume_grade_dict.keys()]]

fig2.add_trace(
    go.Table(
      header=dict(
          values=header_list,
          line_color=linecolor,
          fill_color=header_color,
          align='right'),
    cells=dict(
        values=[list(liquid_volume_grade_dict.keys()), # 1st column
                list(liquid_volume_grade_dict.values()), #2nd
                list(air_volume_dict.values()),#3rd
                liquid_percents_list,#4th
                air_percents_list],#5th 
          line_color=linecolor,
          fill_color = fill_color,
          align='right')
    ),
    row = 1, col = 2)

#Customising fig aesthetics 



tickvals = initial_grade_percent_list[::2]

fig2.update_xaxes(tickmode = "array", tickvals = tickvals, showspikes=True, spikecolor="green",spikesnap = "cursor", spikemode="across", spikethickness = 2)
fig2.update_yaxes(showspikes=True,spikecolor="green",spikesnap = "cursor", spikemode="across", spikethickness = 2)


fig2.update_layout(
    template = "simple_white", 
    title="Air/Liquid Volume Split at Various Grades",
    height = fig2_height, 
    xaxis_title="Grade (%)",

    yaxis=dict(
    title= "% Pipe Volume",
    side="right", 
    range = [0,100]
  ),
  yaxis2=dict(
    title=f"Pipe Volume ({units_volume})",
    anchor="free",
    overlaying="y",
    side="left", 
  ),
  #Legend info 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="left",
    x=0
))
